In [377]:
import json
from gridappsd.simulation import Simulation # Import Simulation Library
from gridappsd import GridAPPSD
import time
from gridappsd.topics import simulation_input_topic
from gridappsd import topics as t


In [485]:
username = "system"
password = "manager"

# Note: there are other parameters for connecting to
# systems other than localhost
gapps = GridAPPSD(username=username, password=password)


checking for connection to gridappsd

In [486]:
assert gapps.connected

adding the run_config for simulation setup

In [487]:
run123_config = json.load(open("Run123NodeFileSimAPI.json")) # Pull simulation start message from saved file

In [488]:
simulation_obj = Simulation(gapps, run123_config) # Create Simulation object
simulation_obj.start_simulation() # Start Simulation

simulation_id = simulation_obj.simulation_id # Obtain Simulation ID
print("Successfully started simulation with simulation_id: ", simulation_id)

Successfully started simulation with simulation_id:  1722904062


taking simulation id from the sim itself, or externally provided

In [489]:
viz_simulation_id = simulation_id
#viz_simulation_id = 1835390618
topic = simulation_input_topic(viz_simulation_id)


start, stop or resume

In [428]:
ch = int(input())
if ch == 1:
    message = {"command": "pause"}
    gapps.send(topic, message)
elif ch == 2:
    message = {"command": "resume"}
    gapps.send(topic, message)
elif ch == 3:
    message = {"command": "stop"}
    gapps.send(topic, message)

else:
    exit()

3


Taking Measurements 

In [363]:
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_MEASUREMENTS",
    "resultFormat": "JSON",
    "objectType": "LoadBreakSwitch"
}

response_obj = gapps.get_response(t.REQUEST_POWERGRID_DATA, message) # Pass query to PowerGrid Models API
measurements_obj = response_obj["data"]
print(measurements_obj)
global Pos_obj # Define global python dictionary of position measurements
Pos_obj = [k for k in measurements_obj if k['type'] == 'Pos'] # Filter measurements to just switch positions

print(Pos_obj) # Print switch position measurement mRID for first switch

[{'measid': '_3bf31a65-6554-4a8c-86f2-52fbbbb1ecb9', 'type': 'PNV', 'class': 'Analog', 'name': 'LinearShuntCompensator_c83', 'bus': '83', 'phases': 'A', 'eqtype': 'LinearShuntCompensator', 'eqname': 'c83', 'eqid': '_D6C44FF1-BC60-49D3-9438-DFAD1AED0A28', 'trmid': '_6A1D377B-AB36-4434-908F-FDF112D5E523'}, {'measid': '_43f8a657-793e-4f66-94ea-fca90f8e90df', 'type': 'PNV', 'class': 'Analog', 'name': 'LinearShuntCompensator_c83', 'bus': '83', 'phases': 'B', 'eqtype': 'LinearShuntCompensator', 'eqname': 'c83', 'eqid': '_D6C44FF1-BC60-49D3-9438-DFAD1AED0A28', 'trmid': '_6A1D377B-AB36-4434-908F-FDF112D5E523'}, {'measid': '_b52146d9-65c0-4a01-ba93-202c6763f13c', 'type': 'PNV', 'class': 'Analog', 'name': 'LinearShuntCompensator_c83', 'bus': '83', 'phases': 'C', 'eqtype': 'LinearShuntCompensator', 'eqname': 'c83', 'eqid': '_D6C44FF1-BC60-49D3-9438-DFAD1AED0A28', 'trmid': '_6A1D377B-AB36-4434-908F-FDF112D5E523'}, {'measid': '_c2300168-c15c-4203-83da-a5eb737b2947', 'type': 'PNV', 'class': 'Analog'

In [490]:
model_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_MEASUREMENTS",
    "resultFormat": "JSON",
    "objectType": "LoadBreakSwitch"
}

response_obj = gapps.get_response(t.REQUEST_POWERGRID_DATA, message)
measurements_obj = response_obj["data"]
print(measurements_obj)
# Switch position measurements (Pos)
global Pos_obj
Pos_obj = [k for k in measurements_obj if k['type'] == 'Pos']

# Switch phase-neutral-voltage measurements (PNV)
global PNV_obj
PNV_obj = [k for k in measurements_obj if k['type'] == 'PNV']

# Switch volt-ampere apparent power measurements (VA)
global VA_obj
VA_obj = [k for k in measurements_obj if k['type'] == 'VA']

# Switch current measurements (A)
global A_obj
A_obj = [k for k in measurements_obj if k['type'] == 'A']
#print(Pos_obj)

[{'measid': '_70387010-be51-4441-baa5-2e2e4e3bfe92', 'type': 'A', 'class': 'Analog', 'name': 'LoadBreakSwitch_sw1_Current', 'bus': '150r', 'phases': 'A', 'eqtype': 'LoadBreakSwitch', 'eqname': 'sw1', 'eqid': '_7CBC54BB-4A93-410F-AF92-DDA633676AA0', 'trmid': '_1F8A4048-225B-431F-8470-BF55B5446154'}, {'measid': '_8da8d5ff-8b34-44b4-b4e3-b9e7365ccad7', 'type': 'A', 'class': 'Analog', 'name': 'LoadBreakSwitch_sw1_Current', 'bus': '150r', 'phases': 'B', 'eqtype': 'LoadBreakSwitch', 'eqname': 'sw1', 'eqid': '_7CBC54BB-4A93-410F-AF92-DDA633676AA0', 'trmid': '_1F8A4048-225B-431F-8470-BF55B5446154'}, {'measid': '_9b8929dd-043e-4d97-8129-577d1756811c', 'type': 'A', 'class': 'Analog', 'name': 'LoadBreakSwitch_sw1_Current', 'bus': '150r', 'phases': 'C', 'eqtype': 'LoadBreakSwitch', 'eqname': 'sw1', 'eqid': '_7CBC54BB-4A93-410F-AF92-DDA633676AA0', 'trmid': '_1F8A4048-225B-431F-8470-BF55B5446154'}, {'measid': '_4d196563-78bc-4df7-ac03-7663c409ae95', 'type': 'A', 'class': 'Analog', 'name': 'LoadBreak

In [476]:
# Use queryFilter of "startTime" and "endTime"
message = {
    "queryMeasurement":"weather",
    "queryFilter":{"startTime":"1357048800000000",
                   "endTime":"1357048860000000"},
    "responseFormat":"JSON"
}

response_obj = gapps.get_response(t.TIMESERIES, message) # Pass query to Timeseries API
weather_obj = response_obj["data"]

print(weather_obj[1]) # Print first line of weather data

{'Diffuse': 5.1879076, 'AvgWindSpeed': 6.6437, 'TowerRH': 36.9, 'long': '105.18 W', 'MST': '14:01', 'TowerDryBulbTemp': 30.0326, 'DATE': '1/1/2013', 'DirectCH1': 86.69558059999999, 'GlobalCM22': 37.990990499999995, 'AvgWindDirection': 115.4, 'time': 1357048860, 'place': 'Solar Radiation Research Laboratory', 'lat': '39.74 N'}


subscribing to output

In [491]:
def demoSubscription1(header, message):
    # Extract time and measurement values from message
    timestamp = message["message"]["timestamp"]
    meas_value = message["message"]["measurements"]
    #print(meas_value)
    meas_mrid = list(meas_value.keys()) #obtain list of all mrid from message

    for index in PNV_obj:
        if index["measid"] in meas_value:
            mrid = index["measid"]
            name = index["eqname"]
            voltage = meas_value[mrid]
            if (voltage["magnitude"] !=0):
                print("voltage of ", name, "= ",voltage["magnitude"])
    
    for index in A_obj:
        if index["measid"] in meas_value:
            mrid = index["measid"]
            name = index["eqname"]
            voltage = meas_value[mrid]
            if (voltage["magnitude"] !=0):
                print("current of ", name, "= ",voltage["magnitude"])
            


In [492]:

from gridappsd.topics import simulation_output_topic

output_topic = simulation_output_topic(viz_simulation_id)

conn_id1 = gapps.subscribe(output_topic, demoSubscription1)

voltage of  sw1 =  2495.3043596637062
voltage of  sw1 =  2495.314551131807
voltage of  sw1 =  2495.287978785251
voltage of  sw1 =  2495.272760709201
voltage of  sw1 =  2495.258348392218
voltage of  sw1 =  2495.2931971232797
voltage of  sw2 =  2403.830097819168
voltage of  sw2 =  2434.2088793623475
voltage of  sw2 =  2483.959362755612
voltage of  sw2 =  2483.9815289363264
voltage of  sw2 =  2403.8046647346914
voltage of  sw2 =  2434.2318638912266
voltage of  sw3 =  2416.1774232490425
voltage of  sw3 =  2476.5503010384496
voltage of  sw3 =  2416.1804968119377
voltage of  sw3 =  2380.020142986643
voltage of  sw3 =  2476.5466234222004
voltage of  sw3 =  2380.0152141773087
voltage of  sw4 =  2353.3182595792846
voltage of  sw4 =  2394.943037358051
voltage of  sw4 =  2353.34102598284
voltage of  sw4 =  2463.745221156658
voltage of  sw4 =  2394.9632830265855
voltage of  sw4 =  2463.764503993201
voltage of  sw5 =  2524.349999210882
voltage of  sw5 =  2524.3483417672423
voltage of  sw5 =  2501.9

voltage of  sw1 =  2495.3043596637062
voltage of  sw1 =  2495.314551131807
voltage of  sw1 =  2495.287978785251
voltage of  sw1 =  2495.272760709201
voltage of  sw1 =  2495.258348392218
voltage of  sw1 =  2495.2931971232797
voltage of  sw2 =  2403.830097819168
voltage of  sw2 =  2434.2088793623475
voltage of  sw2 =  2483.959362755612
voltage of  sw2 =  2483.9815289363264
voltage of  sw2 =  2403.8046647346914
voltage of  sw2 =  2434.2318638912266
voltage of  sw3 =  2416.1774232490425
voltage of  sw3 =  2476.5503010384496
voltage of  sw3 =  2416.1804968119377
voltage of  sw3 =  2380.020142986643
voltage of  sw3 =  2476.5466234222004
voltage of  sw3 =  2380.0152141773087
voltage of  sw4 =  2353.3182595792846
voltage of  sw4 =  2394.943037358051
voltage of  sw4 =  2353.34102598284
voltage of  sw4 =  2463.745221156658
voltage of  sw4 =  2394.9632830265855
voltage of  sw4 =  2463.764503993201
voltage of  sw5 =  2524.349999210882
voltage of  sw5 =  2524.3483417672423
voltage of  sw5 =  2501.9

voltage of  sw1 =  2495.3312920838703
voltage of  sw1 =  2495.339001482654
voltage of  sw1 =  2495.3338359394425
voltage of  sw1 =  2495.3217538032236
voltage of  sw1 =  2495.3140916816387
voltage of  sw1 =  2495.325682048626
voltage of  sw2 =  2450.4659172901875
voltage of  sw2 =  2490.3395977467612
voltage of  sw2 =  2460.7974503454257
voltage of  sw3 =  2442.598637245594
voltage of  sw3 =  2482.9595694988006
voltage of  sw3 =  2442.6017110650346
voltage of  sw3 =  2426.6983482368214
voltage of  sw3 =  2482.955890380492
voltage of  sw3 =  2426.693417125799
voltage of  sw7 =  2467.1500390108195
voltage of  sw7 =  2405.4496545663806
voltage of  sw7 =  2429.1788829138636
current of  sw1 =  313.18278431597383
current of  sw1 =  163.48109804373698
current of  sw1 =  246.85264854636546
current of  sw3 =  98.19335994004688
current of  sw3 =  126.44613966129505
current of  sw3 =  145.65463701132293
voltage of  sw1 =  2495.3312920838703
voltage of  sw1 =  2495.339001482654
voltage of  sw1 =  

voltage of  sw1 =  2495.3312920838703
voltage of  sw1 =  2495.339001482654
voltage of  sw1 =  2495.3338359394425
voltage of  sw1 =  2495.3217538032236
voltage of  sw1 =  2495.3140916816387
voltage of  sw1 =  2495.325682048626
voltage of  sw2 =  2450.4659172901875
voltage of  sw2 =  2490.3395977467612
voltage of  sw2 =  2460.7974503454257
voltage of  sw3 =  2442.598637245594
voltage of  sw3 =  2482.9595694988006
voltage of  sw3 =  2442.6017110650346
voltage of  sw3 =  2426.6983482368214
voltage of  sw3 =  2482.955890380492
voltage of  sw3 =  2426.693417125799
voltage of  sw7 =  2467.1500390108195
voltage of  sw7 =  2405.4496545663806
voltage of  sw7 =  2429.1788829138636
current of  sw1 =  313.18278431597383
current of  sw1 =  163.48109804373698
current of  sw1 =  246.85264854636546
current of  sw3 =  98.19335994004688
current of  sw3 =  126.44613966129505
current of  sw3 =  145.65463701132293
voltage of  sw1 =  2495.3312920838703
voltage of  sw1 =  2495.339001482654
voltage of  sw1 =  

voltage of  sw1 =  2495.3312920838703
voltage of  sw1 =  2495.339001482654
voltage of  sw1 =  2495.3338359394425
voltage of  sw1 =  2495.3217538032236
voltage of  sw1 =  2495.3140916816387
voltage of  sw1 =  2495.325682048626
voltage of  sw2 =  2450.4659172901875
voltage of  sw2 =  2490.3395977467612
voltage of  sw2 =  2460.7974503454257
voltage of  sw3 =  2442.598637245594
voltage of  sw3 =  2482.9595694988006
voltage of  sw3 =  2442.6017110650346
voltage of  sw3 =  2426.6983482368214
voltage of  sw3 =  2482.955890380492
voltage of  sw3 =  2426.693417125799
voltage of  sw7 =  2467.1500390108195
voltage of  sw7 =  2405.4496545663806
voltage of  sw7 =  2429.1788829138636
current of  sw1 =  313.18278431597383
current of  sw1 =  163.48109804373698
current of  sw1 =  246.85264854636546
current of  sw3 =  98.19335994004688
current of  sw3 =  126.44613966129505
current of  sw3 =  145.65463701132293
voltage of  sw1 =  2495.3312920838703
voltage of  sw1 =  2495.339001482654
voltage of  sw1 =  

...

voltage of  sw1 =  2495.3043596637062
voltage of  sw1 =  2495.314551131807
voltage of  sw1 =  2495.287978785251
voltage of  sw1 =  2495.272760709201
voltage of  sw1 =  2495.258348392218
voltage of  sw1 =  2495.2931971232797
voltage of  sw2 =  2403.830097819168
voltage of  sw2 =  2434.2088793623475
voltage of  sw2 =  2483.959362755612
voltage of  sw2 =  2483.9815289363264
voltage of  sw2 =  2403.8046647346914
voltage of  sw2 =  2434.2318638912266
voltage of  sw3 =  2416.1774232490425
voltage of  sw3 =  2476.5503010384496
voltage of  sw3 =  2416.1804968119377
voltage of  sw3 =  2380.020142986643
voltage of  sw3 =  2476.5466234222004
voltage of  sw3 =  2380.0152141773087
voltage of  sw4 =  2353.3182595792846
voltage of  sw4 =  2394.943037358051
voltage of  sw4 =  2353.34102598284
voltage of  sw4 =  2463.745221156658
voltage of  sw4 =  2394.9632830265855
voltage of  sw4 =  2463.764503993201
voltage of  sw5 =  2524.349999210882
voltage of  sw5 =  2524.3483417672423
voltage of  sw5 =  2501.9

voltage of  sw1 =  2495.3043596637062
voltage of  sw1 =  2495.314551131807
voltage of  sw1 =  2495.287978785251
voltage of  sw1 =  2495.272760709201
voltage of  sw1 =  2495.258348392218
voltage of  sw1 =  2495.2931971232797
voltage of  sw2 =  2403.830097819168
voltage of  sw2 =  2434.2088793623475
voltage of  sw2 =  2483.959362755612
voltage of  sw2 =  2483.9815289363264
voltage of  sw2 =  2403.8046647346914
voltage of  sw2 =  2434.2318638912266
voltage of  sw3 =  2416.1774232490425
voltage of  sw3 =  2476.5503010384496
voltage of  sw3 =  2416.1804968119377
voltage of  sw3 =  2380.020142986643
voltage of  sw3 =  2476.5466234222004
voltage of  sw3 =  2380.0152141773087
voltage of  sw4 =  2353.3182595792846
voltage of  sw4 =  2394.943037358051
voltage of  sw4 =  2353.34102598284
voltage of  sw4 =  2463.745221156658
voltage of  sw4 =  2394.9632830265855
voltage of  sw4 =  2463.764503993201
voltage of  sw5 =  2524.349999210882
voltage of  sw5 =  2524.3483417672423
voltage of  sw5 =  2501.9

voltage of  sw1 =  2495.3043596637062
voltage of  sw1 =  2495.314551131807
voltage of  sw1 =  2495.287978785251
voltage of  sw1 =  2495.272760709201
voltage of  sw1 =  2495.258348392218
voltage of  sw1 =  2495.2931971232797
voltage of  sw2 =  2403.830097819168
voltage of  sw2 =  2434.2088793623475
voltage of  sw2 =  2483.959362755612
voltage of  sw2 =  2483.9815289363264
voltage of  sw2 =  2403.8046647346914
voltage of  sw2 =  2434.2318638912266
voltage of  sw3 =  2416.1774232490425
voltage of  sw3 =  2476.5503010384496
voltage of  sw3 =  2416.1804968119377
voltage of  sw3 =  2380.020142986643
voltage of  sw3 =  2476.5466234222004
voltage of  sw3 =  2380.0152141773087
voltage of  sw4 =  2353.3182595792846
voltage of  sw4 =  2394.943037358051
voltage of  sw4 =  2353.34102598284
voltage of  sw4 =  2463.745221156658
voltage of  sw4 =  2394.9632830265855
voltage of  sw4 =  2463.764503993201
voltage of  sw5 =  2524.349999210882
voltage of  sw5 =  2524.3483417672423
voltage of  sw5 =  2501.9

voltage of  sw1 =  2495.3043596637062
voltage of  sw1 =  2495.314551131807
voltage of  sw1 =  2495.287978785251
voltage of  sw1 =  2495.272760709201
voltage of  sw1 =  2495.258348392218
voltage of  sw1 =  2495.2931971232797
voltage of  sw2 =  2403.830097819168
voltage of  sw2 =  2434.2088793623475
voltage of  sw2 =  2483.959362755612
voltage of  sw2 =  2483.9815289363264
voltage of  sw2 =  2403.8046647346914
voltage of  sw2 =  2434.2318638912266
voltage of  sw3 =  2416.1774232490425
voltage of  sw3 =  2476.5503010384496
voltage of  sw3 =  2416.1804968119377
voltage of  sw3 =  2380.020142986643
voltage of  sw3 =  2476.5466234222004
voltage of  sw3 =  2380.0152141773087
voltage of  sw4 =  2353.3182595792846
voltage of  sw4 =  2394.943037358051
voltage of  sw4 =  2353.34102598284
voltage of  sw4 =  2463.745221156658
voltage of  sw4 =  2394.9632830265855
voltage of  sw4 =  2463.764503993201
voltage of  sw5 =  2524.349999210882
voltage of  sw5 =  2524.3483417672423
voltage of  sw5 =  2501.9

voltage of  sw1 =  2495.3043596637062
voltage of  sw1 =  2495.314551131807
voltage of  sw1 =  2495.287978785251
voltage of  sw1 =  2495.272760709201
voltage of  sw1 =  2495.258348392218
voltage of  sw1 =  2495.2931971232797
voltage of  sw2 =  2403.830097819168
voltage of  sw2 =  2434.2088793623475
voltage of  sw2 =  2483.959362755612
voltage of  sw2 =  2483.9815289363264
voltage of  sw2 =  2403.8046647346914
voltage of  sw2 =  2434.2318638912266
voltage of  sw3 =  2416.1774232490425
voltage of  sw3 =  2476.5503010384496
voltage of  sw3 =  2416.1804968119377
voltage of  sw3 =  2380.020142986643
voltage of  sw3 =  2476.5466234222004
voltage of  sw3 =  2380.0152141773087
voltage of  sw4 =  2353.3182595792846
voltage of  sw4 =  2394.943037358051
voltage of  sw4 =  2353.34102598284
voltage of  sw4 =  2463.745221156658
voltage of  sw4 =  2394.9632830265855
voltage of  sw4 =  2463.764503993201
voltage of  sw5 =  2524.349999210882
voltage of  sw5 =  2524.3483417672423
voltage of  sw5 =  2501.9

unsubscribing from simulation

In [493]:
gapps.unsubscribe(conn_id1)

voltage of  sw1 =  2495.3043596637062
voltage of  sw1 =  2495.314551131807
voltage of  sw1 =  2495.287978785251
voltage of  sw1 =  2495.272760709201
voltage of  sw1 =  2495.258348392218
voltage of  sw1 =  2495.2931971232797
voltage of  sw2 =  2403.830097819168
voltage of  sw2 =  2434.2088793623475
voltage of  sw2 =  2483.959362755612
voltage of  sw2 =  2483.9815289363264
voltage of  sw2 =  2403.8046647346914
voltage of  sw2 =  2434.2318638912266
voltage of  sw3 =  2416.1774232490425
voltage of  sw3 =  2476.5503010384496
voltage of  sw3 =  2416.1804968119377
voltage of  sw3 =  2380.020142986643
voltage of  sw3 =  2476.5466234222004
voltage of  sw3 =  2380.0152141773087
voltage of  sw4 =  2353.3182595792846
voltage of  sw4 =  2394.943037358051
voltage of  sw4 =  2353.34102598284
voltage of  sw4 =  2463.745221156658
voltage of  sw4 =  2394.9632830265855
voltage of  sw4 =  2463.764503993201
voltage of  sw5 =  2524.349999210882
voltage of  sw5 =  2524.3483417672423
voltage of  sw5 =  2501.9

storing the mrid

In [330]:
model_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_DICT",
    "resultFormat": "JSON",
    "objectType": "LoadBreakSwitch"
}

response_obj = gapps.get_response(t.REQUEST_POWERGRID_DATA, message)
switch_dict = response_obj["data"]
# print(switch_dict)
sw_mrid=''

for index in switch_dict:
    if index["IdentifiedObject.name"] == 'sw1':
        sw_mrid = index["IdentifiedObject.mRID"]
print(sw_mrid)

_7CBC54BB-4A93-410F-AF92-DDA633676AA0


publishing inputs to the sim using DifferenceBuilder

In [ ]:
import time
from gridappsd import DifferenceBuilder
input_topic = simulation_input_topic(viz_simulation_id)

model_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507" # model mrid of IEEE123 sim

my_open_diff = DifferenceBuilder(viz_simulation_id)
my_open_diff.add_difference(sw_mrid, "Switch.open", 1, 0) # Open switch given by sw_mrid
open_message = my_open_diff.get_message()
gapps.send(input_topic, open_message)

# time.sleep(10)

# my_close_diff = DifferenceBuilder(viz_simulation_id)
# my_close_diff.add_difference(sw_mrid, "Switch.open", 0, 1) # Close switch given by sw_mrid
# close_message = my_close_diff.get_message()
# gapps.send(input_topic, close_message)


input using mesages

In [ ]:
input_topic = simulation_input_topic(viz_simulation_id)
message = {
  "command": "update",
  "input": {
      "simulation_id": str(viz_simulation_id),
      "message": {
          "timestamp": int(time.time()),
          "difference_mrid": "_abcd1234",
          "reverse_differences": [{

                  "object": sw_mrid,
                  "attribute": "Switch.open",
                  "value": 1
              }
          ],
          "forward_differences": [{

                  "object": sw_mrid,
                  "attribute": "Switch.open",
                  "value": 0
              }
              ]
              }
      }
}
gapps.send(input_topic, message)
time.sleep(1)
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_DICT",
    "resultFormat": "JSON",
    "objectType": "LoadBreakSwitch"
}
response_obj = gapps.get_response(t.REQUEST_POWERGRID_DATA, message)
switch_dict = response_obj["data"]
for index in switch_dict:
    if index["IdentifiedObject.name"] == 'sw1':
        print(index)



inputs for capacitors

In [293]:

message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_DICT",
    "resultFormat": "JSON",
    "objectType": "LinearShuntCompensator"
}
while True:
    time.sleep(2)
    response_obj = gapps.get_response(t.REQUEST_POWERGRID_DATA, message)
#     print(response_obj)
    cap_dict = response_obj["data"]
    for index in cap_dict:
        if index['IdentifiedObject.name'] == 'c92c':
            print(index, index["ShuntCompensator.sections"])



{'id': '_939CA567-AA3D-4972-AABC-1D0AAF4859FE', 'ConductingEquipment.BaseVoltage': '_C26F5029-D0AF-4301-B8D5-7A9767B95DEE', 'Equipment.EquipmentContainer': '_C1C3E687-6FFD-C753-582B-632A27E28507', 'IdentifiedObject.mRID': '_939CA567-AA3D-4972-AABC-1D0AAF4859FE', 'IdentifiedObject.name': 'c92c', 'PowerSystemResource.Location': '_A57616AD-93BB-4B6D-8A7E-743CD74BCE4D', 'LinearShuntCompensator.b0PerSection': '0', 'LinearShuntCompensator.bPerSection': '0.008666106', 'LinearShuntCompensator.g0PerSection': '0', 'LinearShuntCompensator.gPerSection': '0', 'ShuntCompensator.aVRDelay': '0', 'ShuntCompensator.grounded': 'true', 'ShuntCompensator.maximumSections': '1', 'ShuntCompensator.nomU': '2402', 'ShuntCompensator.normalSections': '1', 'ShuntCompensator.phaseConnection': 'PhaseShuntConnectionKind.Y', 'type': 'LinearShuntCompensator'}
{'id': '_939CA567-AA3D-4972-AABC-1D0AAF4859FE', 'ConductingEquipment.BaseVoltage': '_C26F5029-D0AF-4301-B8D5-7A9767B95DEE', 'Equipment.EquipmentContainer': '_C1C3

KeyboardInterrupt: 